# Load libraries

In [1]:
import dowhy
from dowhy import CausalModel
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/heart.csv')

In [3]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
df = df.astype({"exang":'bool'}, copy=False)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,False,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,False,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,False,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,False,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,True,0.6,2,0,2,1


In [6]:
xs = ""
for i in range(1,13):
    xs += ("x"+str(i)+"+")
    
model=CausalModel(
        data = df,
        treatment='exang',
        outcome='target',
        common_causes=xs.split('+')
        )

INFO:dowhy.causal_model:Model to find the causal effect of treatment ['exang'] on outcome ['target']


In [7]:
identified_estimand = model.identify_effect()

INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['', 'x9', 'x11', 'x8', 'x6', 'x7', 'x10', 'x3', 'x1', 'x5', 'x12', 'x4', 'x2']


WARN: Do you want to continue by ignoring any unobserved confounders? (use proceed_when_unidentifiable=True to disable this prompt) [y/n] y


INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]


## Using Linear Regression

In [8]:
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression", test_significance=True
)

print(estimate)

print("Causal Estimate is " + str(estimate.value))
data_1 = df[df["exang"]==1]
data_0 = df[df["exang"]==0]

print("ATE", np.mean(data_1["target"])- np.mean(data_0["target"]))


INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: target~exang


*** Causal Estimate ***

## Target estimand
Estimand type: nonparametric-ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
   d                         
────────(Expectation(target))
d[exang]                     
Estimand assumption 1, Unconfoundedness: If U→{exang} and U→target then P(target|exang,,U) = P(target|exang,)
### Estimand : 2
Estimand name: iv
No such variable found!

## Realized estimand
b: target~exang
## Estimate
Value: -0.4637551990493169

## Statistical Significance
p-value: <0.001

Causal Estimate is -0.4637551990493169
ATE -0.46375519904931667


## Refute

In [11]:
refute_results=model.refute_estimate(identified_estimand, estimate,
        method_name="random_common_cause")
print(refute_results)

INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: target~exang+w_random


Refute: Add a Random Common Cause
Estimated effect:(-0.4637551990493169,)
New effect:(-0.4643280047946674,)



In [12]:
res_placebo=model.refute_estimate(identified_estimand, estimate,
        method_name="placebo_treatment_refuter", placebo_type="permute")
print(res_placebo)

INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: target~placebo


Refute: Use a Placebo Treatment
Estimated effect:(-0.4637551990493169,)
New effect:(0.04634581105169344,)

